<a href="https://colab.research.google.com/github/Viswonathan06/CLSTM-for-SDA/blob/main/Copy_of_downloading_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importing required libraries
Note: I was doing this in the google colab notebook.So mounted by google drive.In case if you are doing this in your local machine, the respective directories are to be changed accordingly

In [3]:
from google.colab import drive
import requests

In [4]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


This below code will download the dataset from URL that was pasted at 'file_url'

In [6]:
#paste the url here
file_url = "https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_09_26_drive_0001/2011_09_26_drive_0001_sync.zip"

r = requests.get(file_url, stream = True)  
  
#change this according ur file extension
with open("/content/gdrive/My Drive/2011_09_26_drive_0001_sync.zip", "wb") as file:  
    for block in r.iter_content(chunk_size = 1024): 
         if block:  
             file.write(block)

In [ ]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
!mkdir -p /drive3
!google-drive-ocamlfuse /drive3


E: Package 'python-software-properties' has no installation candidate


KeyboardInterrupt: ignored

In [5]:
#extracting
from zipfile import ZipFile
with ZipFile("/content/gdrive/MyDrive/data_depth_annotated.zip") as f:
    f.extractall()

KeyboardInterrupt: ignored

In [8]:
#extracting
from zipfile import ZipFile
with ZipFile("/content/gdrive/MyDrive/2011_09_26_drive_0001_sync.zip") as f:
    f.extractall()

In [95]:
import pandas as pd
import cv2
import numpy as np
from sklearn.utils import shuffle
import os
from collections import deque
import copy
import matplotlib
import matplotlib.pyplot as plt
from keras.utils import *

%matplotlib inline

In [9]:
import os
import numpy as np
frames_main = sorted(os.listdir('/content/2011_09_26/2011_09_26_drive_0001_sync/image_02/data'))
i=0
frames_list = []
# for video_name in videos:
#   list_frame = sorted(os.listdir('./training/image_02/%s'%video_name))
#   frames_list.append(list_frame)
  
frames_main

['0000000000.png',
 '0000000001.png',
 '0000000002.png',
 '0000000003.png',
 '0000000004.png',
 '0000000005.png',
 '0000000006.png',
 '0000000007.png',
 '0000000008.png',
 '0000000009.png',
 '0000000010.png',
 '0000000011.png',
 '0000000012.png',
 '0000000013.png',
 '0000000014.png',
 '0000000015.png',
 '0000000016.png',
 '0000000017.png',
 '0000000018.png',
 '0000000019.png',
 '0000000020.png',
 '0000000021.png',
 '0000000022.png',
 '0000000023.png',
 '0000000024.png',
 '0000000025.png',
 '0000000026.png',
 '0000000027.png',
 '0000000028.png',
 '0000000029.png',
 '0000000030.png',
 '0000000031.png',
 '0000000032.png',
 '0000000033.png',
 '0000000034.png',
 '0000000035.png',
 '0000000036.png',
 '0000000037.png',
 '0000000038.png',
 '0000000039.png',
 '0000000040.png',
 '0000000041.png',
 '0000000042.png',
 '0000000043.png',
 '0000000044.png',
 '0000000045.png',
 '0000000046.png',
 '0000000047.png',
 '0000000048.png',
 '0000000049.png',
 '0000000050.png',
 '0000000051.png',
 '0000000052

In [10]:
import os
import numpy as np
import pandas as pd
frames_gt = sorted(os.listdir('/content/train/2011_09_26_drive_0001_sync/proj_depth/groundtruth/image_02'))
i=0
frames_list = []
# for video_name in videos:
#   list_frame = sorted(os.listdir('./training/image_02/%s'%video_name))
#   frames_list.append(list_frame)
  
frames_gt

['0000000005.png',
 '0000000006.png',
 '0000000007.png',
 '0000000008.png',
 '0000000009.png',
 '0000000010.png',
 '0000000011.png',
 '0000000012.png',
 '0000000013.png',
 '0000000014.png',
 '0000000015.png',
 '0000000016.png',
 '0000000017.png',
 '0000000018.png',
 '0000000019.png',
 '0000000020.png',
 '0000000021.png',
 '0000000022.png',
 '0000000023.png',
 '0000000024.png',
 '0000000025.png',
 '0000000026.png',
 '0000000027.png',
 '0000000028.png',
 '0000000029.png',
 '0000000030.png',
 '0000000031.png',
 '0000000032.png',
 '0000000033.png',
 '0000000034.png',
 '0000000035.png',
 '0000000036.png',
 '0000000037.png',
 '0000000038.png',
 '0000000039.png',
 '0000000040.png',
 '0000000041.png',
 '0000000042.png',
 '0000000043.png',
 '0000000044.png',
 '0000000045.png',
 '0000000046.png',
 '0000000047.png',
 '0000000048.png',
 '0000000049.png',
 '0000000050.png',
 '0000000051.png',
 '0000000052.png',
 '0000000053.png',
 '0000000054.png',
 '0000000055.png',
 '0000000056.png',
 '0000000057

In [11]:
train_data_path = os.path.join('content','train')
test_data_path = os.path.join('content','test')

In [12]:
if not os.path.exists('data_files'):
  os.mkdir('data_files')
if not os.path.exists('data_files/train'):
  os.mkdir('data_files/train')
if not os.path.exists('data_files/test'):
  os.mkdir('data_files/test')


In [67]:
train_dir = "/content/2011_09_26/"
data_dir_list = sorted(os.listdir(train_dir))
# print(data_dir_list)
for data_dir in data_dir_list:
  video_dir = (data_dir+"/image_02")
  ground_data_dir_list = sorted(os.listdir("/content/train/"+data_dir+"/proj_depth/groundtruth/image_02"))
  video_data_dir_list = sorted( os.listdir(train_dir+video_dir+"/data"))
  train_df = pd.DataFrame(columns = ['ImagePath','GroundTruthPath','VideoName'])
  
  for img in video_data_dir_list: # looping over every frame within the video
      img_path = os.path.join(train_dir,video_dir,"data",img)
      gt_path = os.path.join("/content/train",data_dir,"proj_depth/groundtruth/image_02",img)
      # print(img_path,gt_path)
      train_df = train_df.append({'ImagePath':img_path,'GroundTruthPath':gt_path,'VideoName':data_dir},ignore_index=True)
  file_name = '{}.csv'.format(data_dir)
  train_df.to_csv('/content/data_files/train/{}'.format(file_name))

os.path.abspath(data_dir_list[0])
  

'/content/2011_09_26_drive_0001_sync'

In [84]:
from collections import deque 

def file_generator(data_path,data_files,temporal_stride=1,temporal_length=16):
    '''
    data_files - list of csv files to be read.
    '''
    for f in data_files: # read all the csv files (one csv file corresponds to one vdieo) in data_files one by one
        tmp_df = pd.read_csv(os.path.join(data_path,f))
        gt_list = list(tmp_df['GroundTruthPath'])  # Load all the labels in the label_list
        total_images = len(gt_list) 
        print(total_images)
        if total_images>=temporal_length: # only if the number of frames in the video is greater tha temporal length, use that video
            num_samples = int((total_images-temporal_length)/temporal_stride)+1
            print ('num of samples from vid seq-{}: {}'.format(f,num_samples))
            img_list = list(tmp_df['ImagePath'])
        else: # if the number of frames are less than temporal length , discard it
            print ('num of frames is less than temporal length; hence discarding this file-{}'.format(f))
            continue

        start_frame = 0
        samples = deque() # initliaze a queue to store the frames 
        groundtruth = deque()
        gtcount=0
        samp_count=0 # a counter to count the number of smaple. one smaple has as many frames as defined by temporal length
        for img,gt_img in zip(img_list,gt_list):
            samples.append(img)
            groundtruth.append(gt_img)
            if len(samples)==temporal_length: #if the queue has as many frames as temporal length, return it as one sample
                samples_c=copy.deepcopy(samples) # copy the queue as in the next stage frames would be popped
                samples_gt = copy.deepcopy(groundtruth)
                samp_count+=1
                gtcount+=1
                for t in range(temporal_stride): # pop out as many frames as described by the stride from the left to accomodate new frames
                    samples.popleft()
                    groundtruth.popleft()
                yield samples_c,samples_gt # return a sample(consisting of as many frames as defined by temporal length) 
                                                # and its corsponding label

In [88]:
def load_samples(data_cat='train',temporal_stride=1,temporal_length=16):
    data_path = os.path.join('/content/data_files',data_cat)
    data_files = os.listdir(data_path)
    # define a generator to read the samples
    file_gen = file_generator(data_path,data_files,temporal_stride,temporal_length)
    iterator = True
    data_list = []
    i=0;
    while iterator:
        try:
            x,y = next(file_gen)
            x=list(x)
            y=list(x)
            # print(x)
            data_list.append([x,y])
            # print("Done", i, "times")
            # i = i+1
        except Exception as e:
            print ('the exception: ',e)
            iterator = False
            print ('end of data generator')
    return data_list

In [165]:
training_data = load_samples(data_cat='train',temporal_stride=1,temporal_length=10)
training_data

108
num of samples from vid seq-2011_09_26_drive_0001_sync.csv: 99
Done 0 times
Done 1 times
Done 2 times
Done 3 times
Done 4 times
Done 5 times
Done 6 times
Done 7 times
Done 8 times
Done 9 times
Done 10 times
Done 11 times
Done 12 times
Done 13 times
Done 14 times
Done 15 times
Done 16 times
Done 17 times
Done 18 times
Done 19 times
Done 20 times
Done 21 times
Done 22 times
Done 23 times
Done 24 times
Done 25 times
Done 26 times
Done 27 times
Done 28 times
Done 29 times
Done 30 times
Done 31 times
Done 32 times
Done 33 times
Done 34 times
Done 35 times
Done 36 times
Done 37 times
Done 38 times
Done 39 times
Done 40 times
Done 41 times
Done 42 times
Done 43 times
Done 44 times
Done 45 times
Done 46 times
Done 47 times
Done 48 times
Done 49 times
Done 50 times
Done 51 times
Done 52 times
Done 53 times
Done 54 times
Done 55 times
Done 56 times
Done 57 times
Done 58 times
Done 59 times
Done 60 times
Done 61 times
Done 62 times
Done 63 times
Done 64 times
Done 65 times
Done 66 times
Done 

[[['/content/2011_09_26/2011_09_26_drive_0001_sync/image_02/data/0000000000.png',
   '/content/2011_09_26/2011_09_26_drive_0001_sync/image_02/data/0000000001.png',
   '/content/2011_09_26/2011_09_26_drive_0001_sync/image_02/data/0000000002.png',
   '/content/2011_09_26/2011_09_26_drive_0001_sync/image_02/data/0000000003.png',
   '/content/2011_09_26/2011_09_26_drive_0001_sync/image_02/data/0000000004.png',
   '/content/2011_09_26/2011_09_26_drive_0001_sync/image_02/data/0000000005.png',
   '/content/2011_09_26/2011_09_26_drive_0001_sync/image_02/data/0000000006.png',
   '/content/2011_09_26/2011_09_26_drive_0001_sync/image_02/data/0000000007.png',
   '/content/2011_09_26/2011_09_26_drive_0001_sync/image_02/data/0000000008.png',
   '/content/2011_09_26/2011_09_26_drive_0001_sync/image_02/data/0000000009.png'],
  ['/content/2011_09_26/2011_09_26_drive_0001_sync/image_02/data/0000000000.png',
   '/content/2011_09_26/2011_09_26_drive_0001_sync/image_02/data/0000000001.png',
   '/content/20

In [170]:
print(len(training_data[2]))

2


In [181]:
# batch_suze=18
indexes = np.arange(len(training_data))
# index = indexes[index*batch_size
#       :(index+1)*batch_size]

# list_IDs_temp = [self.list_IDs[k] for k in index]
for i,_ in enumerate(indexes):
  print(i)
  


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98


['/content/2011_09_26/2011_09_26_drive_0001_sync/image_02/data/0000000002.png', '/content/2011_09_26/2011_09_26_drive_0001_sync/image_02/data/0000000003.png', '/content/2011_09_26/2011_09_26_drive_0001_sync/image_02/data/0000000004.png', '/content/2011_09_26/2011_09_26_drive_0001_sync/image_02/data/0000000005.png', '/content/2011_09_26/2011_09_26_drive_0001_sync/image_02/data/0000000006.png', '/content/2011_09_26/2011_09_26_drive_0001_sync/image_02/data/0000000007.png', '/content/2011_09_26/2011_09_26_drive_0001_sync/image_02/data/0000000008.png', '/content/2011_09_26/2011_09_26_drive_0001_sync/image_02/data/0000000009.png', '/content/2011_09_26/2011_09_26_drive_0001_sync/image_02/data/0000000010.png', '/content/2011_09_26/2011_09_26_drive_0001_sync/image_02/data/0000000011.png']


In [209]:
import keras
class DataGenerator(keras.utils.Sequence):
  def __init__(self,data,batch_size,dim,is_autoencoder,shuffle):
    #Initializing the values
    self.dim = dim
    self.data  = data
    self.batch_size = batch_size
    self.list_IDs = np.arange(len(data))
    self.is_autoencoder = is_autoencoder
    self.shuffle = shuffle
    self.on_epoch_end()
    self.n = 0
    self.max = self.__len__()
  

  def on_epoch_end(self): 
    self.indexes = self.list_IDs  #Load the indexes of the data
    if self.shuffle == True:
      np.random.shuffle(self.indexes)

  def __len__(self): 
    return int(np.floor(len(self.data)/self.batch_size))


  def __getitem__(self, index):
    #Generate batch at position 'index' 
    index = self.indexes[index*self.batch_size
      :(index+1)*self.batch_size]
    #Generate a temporary list of indexes that forms a batch based on  ##the index selected above.
    list_IDs_temp = [self.list_IDs[k] for k in index]
    #Generate batch
    X,y = self.__data_generation(list_IDs_temp)
    return X,y

  def __data_generation(self,list_IDs_temp):
    X_data = []
    y_data = []
    for i,_ in enumerate(list_IDs_temp): #Iterating through each ######################################sequence of frames
      # seq_frames = list(self.data.iloc[i,1])
      # y = list(self.data.iloc[i,2])
      temp_data_list = []
      temp_gt_list = []
      for img,gt in enumerate(self.data[i]):
        try:
          image = cv2.imread(img,0)
          ext_img = cv2.resize(image,self.dim)
          image_gt = cv2.imread(gt,0)
          ext_img_gt = cv2.resize(image_gt,self.dim)

          temp_data_list.append(ext_img)
          temp_gt_list.append(ext_img_gt) 
        except Exception as e: 
          print("Code you'd want to run in case of an exception/err")
        
      X_data.append(temp_data_list)
      y_data.append(temp_gt_list)
    X = np.array(X_data) #Converting list to array
    y = np.array(y_data)
    print("Checking... ",x,y)
    if self.is_autoencoder == True:
      return X, X
    else:
      return X, y

  def __next__(self):
        if self.n >= self.max:
           self.n = 0
        result = self.__getitem__(self.n)
        self.n += 1
        return result

In [210]:
params = {
'batch_size':18,
'dim':(256,256),
'is_autoencoder':False,
'shuffle':True }

df = pd.read_csv("/content/data_files/train/2011_09_26_drive_0001_sync.csv")
X,Y = next(DataGenerator(training_data,**params))


Code you'd want to run in case of an exception/err
Code you'd want to run in case of an exception/err
Code you'd want to run in case of an exception/err
Code you'd want to run in case of an exception/err
Code you'd want to run in case of an exception/err
Code you'd want to run in case of an exception/err
Code you'd want to run in case of an exception/err
Code you'd want to run in case of an exception/err
Code you'd want to run in case of an exception/err
Code you'd want to run in case of an exception/err
Code you'd want to run in case of an exception/err
Code you'd want to run in case of an exception/err
Code you'd want to run in case of an exception/err
Code you'd want to run in case of an exception/err
Code you'd want to run in case of an exception/err
Code you'd want to run in case of an exception/err
Code you'd want to run in case of an exception/err
Code you'd want to run in case of an exception/err
Code you'd want to run in case of an exception/err
Code you'd want to run in case 

[]
